# Q2(i)

In [152]:
import pandas as pd
import numpy as np

In [2]:
%run part_1.ipynb


#Entity in gold data: 229
#Entity in prediction: 1466

#Correct Entity : 178
Entity  precision: 0.1214
Entity  recall: 0.7773
Entity  F: 0.2100

#Correct Sentiment : 97
Sentiment  precision: 0.0662
Sentiment  recall: 0.4236
Sentiment  F: 0.1145

#Entity in gold data: 389
#Entity in prediction: 1816

#Correct Entity : 266
Entity  precision: 0.1465
Entity  recall: 0.6838
Entity  F: 0.2413

#Correct Sentiment : 129
Sentiment  precision: 0.0710
Sentiment  recall: 0.3316
Sentiment  F: 0.1170


In [3]:
# Write a function that estimates the transition parameters from the training set using MLE (maximum likelihood estimation)
# and add the transition parameters to the HMM model.

In [4]:
def calculate_transition_probabilities(training_filepath):
    # Open the training file
    with open(training_filepath, "r", encoding="UTF-8") as file:
        # Read the entire contents of the file
        training_contents = file.read().split("\n")
        
    training_labels = []
    # Extract all the lables in a particular order, leaving the spaces as 2 labels "STOP" followed by "START"          
    for line in training_contents:
        # Exclude spaces in the dataset in training
        if len(line) > 0:
            parts = line.split()  # Split the line by spaces
            # To account for RUS cases wher there are spaces in the words
            if len(parts) >= 3:    
                label = parts[-1]            # The last part is the label
                training_labels.append(label)
            else:
                training_labels.append(parts[1])
        else:
            training_labels.append("STOP")
            training_labels.append("START")
            
    # Adding START to the first word and removing START from the last word and removing repeat START/STOPS
    training_labels.insert(0, "START")
    training_labels.pop()
    training_labels.pop()
    training_labels.pop()
    
    # Now to calculate the transition probabiltiies of states START, (given 7 states), STOP
    transition_probabilities = {}

    for i in range(len(training_labels)):
        if i < len(training_labels)-1:
            if training_labels[i] not in transition_probabilities:
                transition_probabilities[training_labels[i]] = {}
            
            # Check the next word and add it as a count
            if training_labels[i+1] not in transition_probabilities[training_labels[i]]:
                transition_probabilities[training_labels[i]][training_labels[i+1]] = 1
            else:
                transition_probabilities[training_labels[i]][training_labels[i+1]] += 1
                
    # Calculate the transition probabilities from above
    for outside_label in transition_probabilities:
        total_count = sum(transition_probabilities[outside_label].values())
        for inside_label in transition_probabilities[outside_label]:
            transition_probabilities[outside_label][inside_label] = transition_probabilities[outside_label][inside_label] / total_count
    
    return transition_probabilities
        
    

In [5]:
# Transition Parameters
ES_transition_probabilities = calculate_transition_probabilities(training_filepath="Data/ES/train")
RU_transition_probabilties = calculate_transition_probabilities(training_filepath="Data/RU/train")

# Emission Parameters
ES_emission_probabilities = ES_training_emission_params_v2
RU_emission_probabilities = RU_training_emission_params_v2

In [6]:
ES_emission_probabilities

{'O': {',': 0.05730816916930707,
  '.': 0.055896128943380634,
  'y': 0.0352665656426505,
  'de': 0.03481884557101529,
  'que': 0.029101804656288744,
  'la': 0.026002204160352666,
  'el': 0.022110483537677365,
  'en': 0.017392202782752445,
  'a': 0.014774762363961979,
  'muy': 0.013638242182118749,
  'no': 0.012708362033337925,
  'un': 0.012570602011296322,
  'con': 0.011055241768838682,
  'es': 0.010469761675161868,
  'lo': 0.009298801487808239,
  'por': 0.00864444138311062,
  'los': 0.008506681361069017,
  'una': 0.007886761261881802,
  'pero': 0.006578041052486569,
  'para': 0.006578041052486569,
  'bien': 0.005682600909216145,
  'se': 0.00547596087615374,
  '!': 0.00544152087064334,
  'El': 0.005372640859622538,
  'La': 0.005097120815539331,
  'nos': 0.005028240804518529,
  'me': 0.0049938007990081275,
  'calidad': 0.004924920787987326,
  'buena': 0.00468384074941452,
  ')': 0.004511640721862516,
  '(': 0.004511640721862516,
  'las': 0.004408320705331313,
  'como': 0.004236120677779

In [7]:
ES_transition_probabilities

{'START': {'O': 0.9289176090468497,
  'B-positive': 0.052234787291330104,
  'B-negative': 0.014001077005923533,
  'B-neutral': 0.004846526655896607},
 'O': {'O': 0.8856896848630963,
  'B-positive': 0.03650766316514551,
  'STOP': 0.06344067504735663,
  'B-negative': 0.012226623041157224,
  'B-neutral': 0.0021353538832443605},
 'B-positive': {'O': 0.871551724137931,
  'I-positive': 0.11637931034482758,
  'STOP': 0.008620689655172414,
  'B-neutral': 0.0008620689655172414,
  'B-positive': 0.002586206896551724},
 'STOP': {'START': 1.0},
 'B-negative': {'O': 0.8110236220472441,
  'I-negative': 0.1784776902887139,
  'STOP': 0.010498687664041995},
 'B-neutral': {'I-neutral': 0.20833333333333334, 'O': 0.7916666666666666},
 'I-neutral': {'I-neutral': 0.6511627906976745, 'O': 0.3488372093023256},
 'I-positive': {'I-positive': 0.5700636942675159,
  'O': 0.4267515923566879,
  'STOP': 0.0031847133757961785},
 'I-negative': {'O': 0.39766081871345027, 'I-negative': 0.6023391812865497}}

# 2(ii)

In [8]:
# Implement the Viterbi algorithm

def viterbi_algorithm(transition_probabilties, emission_probabilities, test_sentence):
    # Initalise Viterbi matrix
    
    # Create a N by T matrix where N is the number of tags and T is the length of the test sentence
    viterbi_matrix = np.zeros((len(transition_probabilties), len(test_sentence)+2))
    predicted_sequence = []
    
    
    # Assign an index in the row for each tag
    tag_index = {
        0: 'START',
        1: 'B-positive',
        2: 'B-neutral',
        3: 'B-negative',
        4: 'I-positive',
        5: 'I-neutral',
        6: 'I-negative',
        7: 'O',
        8: 'STOP'
    }
    
    # Initialise with START
    viterbi_matrix[0,0] = 1
        
    # Viterbi Algorithm to populate the matrix
    for j in range(0,len(test_sentence)):
        # First check if word does exist in your test set, else word will be #UNK#
        exists = False
        for label, word_emission_probs in emission_probabilities.items():
                if test_sentence[j] in word_emission_probs:
                    exists = True
        
        
        if j > 1:
            for i in range(1,len(transition_probabilties)-1):
                max_prob = 0
                for k in range(1,len(transition_probabilties)-1):
                    # Calculate all of the probabilities when reaching the current state
                    if tag_index[i] not in transition_probabilties[tag_index[k]]:
                        # Transition from k to i does not exist, probability is 0
                        prob = 0
                    elif test_sentence[j] not in emission_probabilities[tag_index[i]] and not exists:
                        # Word does not exist in test set, emission probability is #UNK#
                        prob = viterbi_matrix[k][j] * emission_probabilities[tag_index[i]]["#UNK#"] * transition_probabilties[tag_index[k]][tag_index[i]]
                    elif test_sentence[j] not in emission_probabilities[tag_index[i]] and exists:
                        # Word exists in test set but does not belong to the current tag, emission probability is 0
                        prob = 0
                    else:
                        # Word exists in test set and also belongs to the current tag
                        prob = viterbi_matrix[k][j] * emission_probabilities[tag_index[i]][test_sentence[j]] * transition_probabilties[tag_index[k]][tag_index[i]]
                    
                    if prob>max_prob:
                        max_prob = prob
                        
                viterbi_matrix[i,j+1] = max_prob
                
                
        # For the first word j=1 immediately after start
        else:
                for i in range(1,len(transition_probabilties)-1):
                    if tag_index[i] not in transition_probabilties['START']:
                        # Transition from START to i does not exist, probability is 0
                        max_prob = 0
                    elif test_sentence[j] not in emission_probabilities[tag_index[i]] and not exists:
                        # Word does not exist in test set, emission probability is #UNK#
                        max_prob = viterbi_matrix[0][0] * transition_probabilties['START'][tag_index[i]] * emission_probabilities[tag_index[i]]["#UNK#"]
                    elif test_sentence[j] not in emission_probabilities[tag_index[i]] and exists:
                        # Word exists in test set but does not belong to the current tag, emission probability is 0
                        max_prob = 0
                    else:
                        # Word exists in test set and also belongs to the current tag
                        max_prob = viterbi_matrix[0][0] * transition_probabilties['START'][tag_index[i]] * emission_probabilities[tag_index[i]][test_sentence[j]]
                    
                    viterbi_matrix[i,j+1] = max_prob
    
    # Final Step
    max_final_prob = 0
    for i in range(0,len(transition_probabilties)):
        if "STOP" not in transition_probabilties[tag_index[i]]:
            prob = 0
        else:
            prob = viterbi_matrix[i][len(test_sentence)] * transition_probabilties[tag_index[i]]['STOP']
            
        if prob>max_final_prob:
            max_final_prob = prob
    
    viterbi_matrix[8][len(test_sentence)+1] = max_final_prob

    
    # print(viterbi_matrix)
        
    # To find the predicted sequence, we need to find the highest value of each column and its index
    for j in range(0,len(test_sentence)+2):
        if j == 0:
            predicted_sequence.append("START")
        elif j == len(test_sentence)+1:
            predicted_sequence.append("STOP")
        else:
            max_prob = 0
            max_index = 0
            for i in range(0,len(transition_probabilties)):
                if viterbi_matrix[i][j] > max_prob:
                    max_prob = viterbi_matrix[i][j]
                    max_index = i
            predicted_sequence.append(tag_index[max_index])
    
    # print(viterbi_matrix)
    return predicted_sequence
    

In [9]:
viterbi_algorithm(ES_transition_probabilities, ES_emission_probabilities, ["Plato", "degustación", ":", "un",
"poco",
"abundante",
"de"])

NameError: name 'np' is not defined

In [ ]:
# Evaluation
def predict(testing_filepath, output_filepath, emission_parameters, transition_parameters):
    with open(testing_filepath, "r", encoding="UTF-8") as file:
        # Read the entire contents of the file
        # test_words_with_spaces = file.read().split('\n')
        test_words =  file.read().split("\n")
    
    # Make sentences out of the words
    test_sentences = []
    temp_sentence_holder = []
    for word in test_words:
        if len(word) == 0:
            if temp_sentence_holder:
                test_sentences.append(temp_sentence_holder)
                temp_sentence_holder = []
        else:
            temp_sentence_holder.append(word)
    
    if temp_sentence_holder:
        test_sentences.append(temp_sentence_holder)
    
    # print(test_sentences)
    
    # Now I will use Viterbi algorithm to predict the tags for each sentence, removing START and STOP tags
    predicted_tags_set = []
    for sentence in test_sentences:
        predicted_tags_set.append(viterbi_algorithm(transition_probabilties=transition_parameters, emission_probabilities=emission_parameters, test_sentence=sentence)[1:-1])
        
    
    # Next I will repack the tags into the format required by the assignment
    predicted_tags_with_spaces = []
    for tags in predicted_tags_set:
        for tag in tags:
            predicted_tags_with_spaces.append(tag)
        predicted_tags_with_spaces.append("")
        
    # Now I will produce the output file
    with open(output_filepath, "w", encoding="UTF-8") as file:
        for i in range(len(test_words)):
            try :
                file.write(test_words[i] + " " + predicted_tags_with_spaces[i] + "\n")
            except:
                file.write("")


In [10]:
predict("Data/ES/dev.in","Data/ES/dev.p2.out", ES_emission_probabilities, ES_transition_probabilities)
predict("Data/RU/dev.in","Data/RU/dev.p2.out", RU_emission_probabilities, RU_transition_probabilties)

NameError: name 'predict' is not defined

In [11]:
# Evaluation of ES
!python EvalScript/evalResult.py Data/ES/dev.out Data/ES/dev.p2.out


#Entity in gold data: 229
#Entity in prediction: 651

#Correct Entity : 141
Entity  precision: 0.2166
Entity  recall: 0.6157
Entity  F: 0.3205

#Correct Sentiment : 103
Sentiment  precision: 0.1582
Sentiment  recall: 0.4498
Sentiment  F: 0.2341


In [12]:
# Evaluation of RU
!python EvalScript/evalResult.py Data/RU/dev.out Data/RU/dev.p2.out


#Entity in gold data: 389
#Entity in prediction: 598

#Correct Entity : 200
Entity  precision: 0.3344
Entity  recall: 0.5141
Entity  F: 0.4053

#Correct Sentiment : 137
Sentiment  precision: 0.2291
Sentiment  recall: 0.3522
Sentiment  F: 0.2776
